In [2]:
!pip install catboost

import pandas as pd
import numpy as np
import sklearn
from sklearn import preprocessing
from sklearn import metrics
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor, CatBoostClassifier

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#Импорт файлов
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

test = pd.read_csv("/content/drive/MyDrive/sample_solution.csv")
train = pd.read_csv("/content/drive/MyDrive/train.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
targets = ["time_to_under","label"]
namelist = list(train.columns.values)

X_train = train
X_train = X_train.drop(targets[0], axis = 1)
X_train = X_train.drop(targets[1], axis = 1) #удаляем timetounder and label from X

Y1_train = train[targets[0]].copy() #тут time to under
Y2_train = train[targets[1]].copy() #тут label



In [5]:
test.isnull().sum()



id               0
time_to_under    0
label            0
dtype: int64

In [24]:
le_ticket_type = preprocessing.LabelEncoder()
le_ticket_id = preprocessing.LabelEncoder()

le_ticket_type.fit(X_train["ticket_type_nm"])
le_ticket_id.fit(X_train["ticket_id"])

def X_preprocessing(X):
    def cut_day(string):
        string = string[8:10]
        return string

    def cut_hour(string):
        string = string[11:13]
        return string

    def cut_minute(string):
        string = string[14:16]
        return string
    
    X = X.drop("entrance_nm", axis = 1) #Т.к. id и имя это один и тот же признак
    X = X.drop("station_nm", axis = 1)
    X = X.drop("line_nm", axis = 1)
    
    X["ticket_type_nm"] = le_ticket_type.transform(X_train["ticket_type_nm"])
    X["ticket_id"] = le_ticket_id.transform(X_train["ticket_id"])
    
#     Заменяем столбец времени на столбцы час и минута
    time_column = X["pass_dttm"].copy()
    hour = time_column.apply(cut_hour)
    day = time_column.apply(cut_day)
    minute = time_column.apply(cut_minute)
    
    hour = pd.to_numeric(hour, errors='coerce')
    day = pd.to_numeric(day, errors='coerce')
    minute = pd.to_numeric(minute, errors='coerce')
    
    
    X = X.drop("pass_dttm", axis = 1)
    X["day"] = day
    X["hour"] = hour
    X["minute"] = minute
    return X

X_preprocessed = X_preprocessing(X_train)
X_preprocessed.head()

,id,ticket_id,ticket_type_nm,entrance_id,station_id,line_id,day,hour,minute
0,1,84909,40,2402,11007,11,12,5,0
1,2,24211,53,110,2006,2,12,5,0
2,3,276072,26,110,2006,2,12,5,0
3,4,1788,3,110,2006,2,12,5,1
4,5,196752,38,110,2006,2,12,5,2


In [25]:
model_reg = CatBoostRegressor(iterations = 10)
model_reg.fit(X_preprocessed,Y1_train, cat_features=['ticket_type_nm','entrance_id','station_id','line_id'])
Y1_pred = model_reg.predict(X_preprocessed)

Learning rate set to 0.5
0:	learn: 186.8410220	total: 295ms	remaining: 2.65s
1:	learn: 169.0537326	total: 582ms	remaining: 2.33s
2:	learn: 163.4953244	total: 824ms	remaining: 1.92s
3:	learn: 161.6055484	total: 1.11s	remaining: 1.67s
4:	learn: 160.7974588	total: 1.36s	remaining: 1.36s
5:	learn: 160.4639809	total: 1.6s	remaining: 1.06s
6:	learn: 160.2609399	total: 1.85s	remaining: 792ms
7:	learn: 160.0723757	total: 2.12s	remaining: 529ms
8:	learn: 159.8120883	total: 2.36s	remaining: 262ms
9:	learn: 159.7092168	total: 2.58s	remaining: 0us


In [27]:
# Теперь классификация
X_preprocessed_cut = X_preprocessed[:500]
Y2_train = Y2_train[:500]
print(X_preprocessed_cut.size, Y2_train.size)


4500 500


In [19]:
print(X_preprocessed)

      id  ticket_id  ticket_type_nm  entrance_id  station_id  line_id  day  \
0      1      84909              40         2402       11007       11   12   
1      2      24211              53          110        2006        2   12   
2      3     276072              26          110        2006        2   12   
3      4       1788               3          110        2006        2   12   
4      5     196752              38          110        2006        2   12   
..   ...        ...             ...          ...         ...      ...  ...   
495  496      28993               6         2086       10001       10   12   
496  497       2148               5          213        5003        5   12   
497  498     281075               5          153        3022        3   12   
498  499      17903              19          188        3006        3   12   
499  500      96578              19          386        9010        9   12   

     hour  minute  
0       5       0  
1       5       0  
2  

In [28]:
model_class = CatBoostClassifier(iterations = 10, depth = 5)
model_class.fit(X_preprocessed_cut,Y2_train, cat_features=['ticket_type_nm','entrance_id','station_id','line_id'])
Y2_pred = model_class.predict(X_preprocessed_cut)

Learning rate set to 0.5
0:	learn: 5.0662030	total: 2.62s	remaining: 23.5s
1:	learn: 4.9204615	total: 5.22s	remaining: 20.9s
2:	learn: 4.7861461	total: 7.79s	remaining: 18.2s
3:	learn: 4.6590857	total: 10.4s	remaining: 15.6s
4:	learn: 4.5662683	total: 13s	remaining: 13s
5:	learn: 4.4750667	total: 15.6s	remaining: 10.4s
6:	learn: 4.3632652	total: 18.1s	remaining: 7.78s
7:	learn: 4.2485552	total: 20.8s	remaining: 5.19s
8:	learn: 4.1488281	total: 23.5s	remaining: 2.61s
9:	learn: 4.0496357	total: 26s	remaining: 0us


In [31]:
from sklearn.metrics import r2_score, recall_score
def result(Y1_train, Y1_pred, Y2_train, Y2_pred):
  r2 = r2_score(Y1_train, Y1_pred)
  rec = recall_score(Y2_train, Y2_pred, average='micro')
  res = 0.5*r2 + 0.5*rec
  return res

In [33]:
res = result(Y1_train, Y1_pred, Y2_train, Y2_pred)
print(res)

0.30084301580547773
